## Module submission header
### Submission preparation instructions 
_Completion of this header is mandatory, subject to a 2-point deduction to the assignment._ Only add plain text in the designated areas, i.e., replacing the relevant 'NA's. You must fill out all group member Names and Drexel email addresses in the below markdown list, under header __Module submission group__. It is required to fill out descriptive notes pertaining to any tutoring support received in the completion of this submission under the __Additional submission comments__ section at the bottom of the header. If no tutoring support was received, leave NA in place. You may as well list other optional comments pertaining to the submission at bottom. _Any distruption of this header's formatting will make your group liable to the 2-point deduction._

### Module submission group
- Group member 1
    - Name: Rishabh Sharma
    - Email: rs3738@drexel.edu
- Group member 2
    - Name: Shai Wudkwych
    - Email: sw3468@drexel.edu

### Additional submission comments
- Tutoring support received: NA
- Other (other): NA

# Assignment group 1: Textual feature extraction and numerical comparison

## Module C _(35 points)_ Similarity of word usage across a document

Here we'll be building up some code to discover how different terms are utilized similarly across a document. For this, our first task will be to create a word frequency counting function.

__C1.__ _(12 points)_ Define a function called `count_words(paragraph, pos = True, lemma = True)` that `return`s a `Counter()` called `frequency`. In `frequency`, each key will consist of a `heading = (text, tag)`, where `text` contains the `word.text` attribute from `spacy` if `lemma = False`, and `word.lemma_` attribute if `True`. Similarly, `tag` should be left empty as `""` if `pos = False` and otherwise contain `word.pos_`. The `Counter()` should simply contain the number of times each `heading` is observed in the `paragraph`.

In [152]:
# C1:Function(12/12)

from collections import Counter
import spacy, json, re

nlp = spacy.load("en_core_web_sm")

def count_words(paragraph, pos = True, lemma = True):
    #---Your code starts here

    frequency = Counter()
    doc = nlp(paragraph)
    for word in doc:
        if lemma == True:
            text = word.lemma_
        else:
            text = word.text
        if pos == True:
            tag = word.pos_
        else:
            tag = ""
        heading = (text, tag)
        frequency[heading] += 1

    #---Your code ends here
    
    return frequency

Let's make sure your function works by testing it on a short sentence. 

In [153]:
# C1:SanityCheck

count_words("The quick brown fox jumps over the lazy dog.")

Counter({('the', 'DET'): 2,
         ('quick', 'ADJ'): 1,
         ('brown', 'ADJ'): 1,
         ('fox', 'NOUN'): 1,
         ('jump', 'VERB'): 1,
         ('over', 'ADP'): 1,
         ('lazy', 'ADJ'): 1,
         ('dog', 'NOUN'): 1,
         ('.', 'PUNCT'): 1})

__C2.__ _(8 pts)_ Next, define a function called `book_TDM(book_id, pos = True, lemma = True)` and copy into it the TDM-producing code from __Section 2.1.5.1__ of the lecture notes, now `return`-ing `TDM` and `all_words`. Once copied, modify this function to call `count_words` appropriately, now passing through the user of `book_TDM`'s specified `lemma` and `pos` arguments.

In [158]:
# C2:Function(8/8)

import numpy as np
from collections import Counter
import re

def book_TDM(book_id, pos = True, lemma = True):

    #---Your code starts here---
    ##  define a function called `book_TDM(book_id, pos = True, lemma = True)` and copy into it the TDM-producing code from __Section 2.1.5.1__ of the lecture notes, now `return`-ing `TDM` and `all_words`. Once copied, modify this function to call `count_words` appropriately, now passing through the user of `book_TDM`'s specified `lemma` and `pos` arguments.
    book = book = './data/books/'+book_id+'.txt'
    with open(book, 'r') as f:
        text = f.read()
    ## breaking the document into the paragraph
    paragraphs = re.split('\n\n+', text)
    all_words = set()
    all_doc_frequencies = {}
    for j, paragraph in enumerate (paragraphs):
        doc_frequencies = count_words(paragraph, pos = pos, lemma = lemma)
        all_doc_frequencies[j] = doc_frequencies
        all_words.update(doc_frequencies.keys())
    all_words = sorted(all_words)
    TDM = np.zeros((len(all_words), len(all_doc_frequencies)))
    for j in all_doc_frequencies:
        for i, word in enumerate(all_words):
            TDM[i,j] = all_doc_frequencies[j][word]
            
    #---Your code ends here---

    return(TDM, all_words)


To test your code's function, let's process `book_id = 84` with both of `pos = True` and `lemma = True` and print out the `TDM`'s `.shape` attribute and the first ten terms in `all_words`.

In [159]:
# C2:SanityCheck

TDM, terms = book_TDM("84", pos = True, lemma = True)
terms[:10]

[('\n', 'SPACE'),
 ('\n  ', 'SPACE'),
 ('\n   ', 'SPACE'),
 ('\n     ', 'SPACE'),
 ('\n                              ', 'SPACE'),
 (' ', 'SPACE'),
 ('  ', 'SPACE'),
 ('    ', 'SPACE'),
 ('     ', 'SPACE'),
 ('               ', 'SPACE')]

In [160]:
# C2:SanityCheck

TDM.shape

(6262, 725)

__C3.__ _(8 pts)_ Next, your job is to define two functions. The first is `sim(u,v)`, which shoud take two arbitrary numeric vectors and compute/output the `cosine_similarity`, as described in __Section 1.1.2.10__.  

The second function is `term_sims(i, TDM)`, which should utilize the first function (`sim` function) to output a list of cosine similarity values (`sim_values`) between the word/row `i` and all others (rows) in the `TDM`.

Note: each of these functions can be straightforwardly completed using a single line of code! Exhibit your knowledge of comprehensions and vectorization!

In [162]:
# C3:Function(4/8)

import numpy as np
from scipy.linalg import norm

def sim(u,v):
    
    #---Your code starts here
    dot = np.dot(u,v)
    norm_u = norm(u)
    norm_v = norm(v)
    # if norm_u == 0 or norm_v == 0:
    #     cosine_similarity = "undefined"
    # else:
    cosine_similarity = dot / (norm_u * norm_v)
    #---Your code ends here
    
    return cosine_similarity

In [163]:
# C3:SanityCheck

print("Exactly similar:", sim(np.array([1,2,3]), np.array([1,2,3])))
print("Exactly dissimilar:", sim(np.array([1,2,3]), np.array([-1,-2,-3])))
print("In the middle:", sim(np.array([1,1]), np.array([-1,1])))

Exactly similar: 1.0
Exactly dissimilar: -1.0
In the middle: 0.0


In [164]:
# C3:Function(4/8)

def term_sims(i, TDM):
    #---Your code starts here
    sim_values = []
    for j in TDM:
        sim_values.append(sim(TDM[i], j))
    #---Your code ends here
    
    return sim_values

In [166]:
# C3:SanityCheck

# Compare word/row 0 to all other (rows) in the TDM
term_sims(0, TDM)

[1.000000000000004,
 0.0,
 0.0,
 0.0,
 0.0,
 0.9498986502907766,
 0.04594929688732405,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.01712429189306268,
 0.3981325871825905,
 0.4674241572909668,
 0.24199963027324,
 0.11127332811291053,
 0.4329628779238967,
 0.2063477173114053,
 0.2063477173114053,
 0.9523276661400802,
 0.1477261756313498,
 0.0,
 0.5274414795006913,
 0.02739886702890029,
 0.41119037303226474,
 0.023974008650287752,
 0.0,
 0.05822259243641311,
 0.9712507693137752,
 0.0,
 0.0,
 0.0,
 0.04109830054335043,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.010274575135837608,
 0.0,
 0.0,
 0.0,
 0.013699433514450144,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.015818743254626344,
 0.0,
 0.0,
 0.04109830054335043,
 0.3558058600387067,
 0.8189613459883727,
 0.3762721959394888,
 0.0,
 0.06920700173899026,
 0.0753468843294758,
 0.14106638902501845,
 0.16510813272016248,
 0.003424858378612536,
 0.027682800695596102,
 0.09491245952775806,
 

__C4.__ _(7 pts)_ Finally, your goal now is to a write function, `most_similar(term, terms, TDM, top = 25)`, that utilizes `term_sims` to output a sorted list of the `top = N` terms (`top_n_terms`) most similar to one specified (`term`). The output data type should be a list of lists, with each inner list representing information for a similar term as: `[row_ix, similarity, term]`. 

\[Hint: to locate the row containing the term of interest, utilize the list `.index()` method in application to the `terms` argument.\]

In [167]:
# C4:Function(6/7)

def most_similar(term, terms, TDM, top = 25):
    #---Your code starts here---
    term_index = terms.index(term)
    sim_values = term_sims(term_index, TDM)
    top_n_terms = []
    for i in np.argsort(sim_values)[::-1][:top]:
        top_n_terms.append(terms[i])
        
    #---Your code ends here---
    
    return top_n_terms

Now, let's test your functions utility on a `TDM` produced for `book_id = 84` and exhibit the top 25 similar terms to both of `('monster', 'NOUN')` and `('beautiful', 'ADJ')`.

In [168]:
# C4:SanityCheck

most_similar(('monster', 'NOUN'), terms, TDM, top = 25)

[('monster', 'NOUN'),
 ('asseveration', 'NOUN'),
 ('posterity', 'NOUN'),
 ('correct', 'VERB'),
 ('mutilate', 'VERB'),
 ('formation', 'NOUN'),
 ('I', 'PRON'),
 ('hideous', 'ADJ'),
 ('demoniacal', 'ADJ'),
 ('forgetfulness', 'NOUN'),
 (' ', 'SPACE'),
 ('that', 'SCONJ'),
 ('!', 'PUNCT'),
 ('\n', 'SPACE'),
 ('.', 'PUNCT'),
 ('the', 'DET'),
 ('and', 'CCONJ'),
 ('neck', 'NOUN'),
 ('loathsome', 'ADJ'),
 ('narrative', 'NOUN'),
 (',', 'PUNCT'),
 ('but', 'CCONJ'),
 (';', 'PUNCT'),
 ('from', 'ADP'),
 ('my', 'PRON')]

In [169]:
# C4:SanityCheck

most_similar(('beautiful', 'ADJ'), terms, TDM, top = 25)

[('beautiful', 'ADJ'),
 ('beneath', 'ADV'),
 ('castle', 'NOUN'),
 ('Rotterdam', 'PROPN'),
 ('singularly', 'ADV'),
 ('horrid', 'ADJ'),
 ('meandering', 'ADJ'),
 ('pearly', 'ADJ'),
 ('willowy', 'PROPN'),
 ('delineate', 'ADJ'),
 ('fifth', 'ADJ'),
 ('whence', 'ADP'),
 ('whiteness', 'NOUN'),
 ('luxuriance', 'NOUN'),
 ('populous', 'ADJ'),
 ('Mainz', 'PROPN'),
 ('dun', 'PROPN'),
 ('straight', 'ADJ'),
 ('shrivel', 'VERB'),
 ('variegate', 'VERB'),
 ('shipping', 'NOUN'),
 ('steep', 'ADJ'),
 ('Mannheim', 'PROPN'),
 ('vineyard', 'NOUN'),
 ('bank', 'NOUN')]

In [124]:
# C4:Inline

# Comment on the ordered results returned in the sanity checks.
# Do you think the algorithm is exhibiting sensible results? print "Yes" or "No"
print("Yes")

No
